## Setup

In [1]:
import re
import json
import pickle
import os
import sys
import requests
import logging
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from tqdm.auto import tqdm
import plotly.io as pio
import numpy as np
import random
import torch.nn as nn
import torch.nn.functional as F
import wandb
import plotly.express as px
import pandas as pd
import torch.nn.init as init
from pathlib import Path
from jaxtyping import Int, Float
from torch import Tensor
import einops
from collections import Counter
from datasets import load_dataset
import pandas as pd
from ipywidgets import interact, IntSlider
from process_tiny_stories_data import load_tinystories_validation_prompts, load_tinystories_tokens
from typing import Literal
from transformer_lens.utils import test_prompt
import pickle
from ipywidgets import interact, IntSlider, SelectionSlider
from transformer_lens.utils import test_prompt

import plotly.graph_objects as go

pio.renderers.default = "notebook_connected"
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
torch.autograd.set_grad_enabled(False)
torch.set_grad_enabled(False)

logging.basicConfig(format='(%(levelname)s) %(asctime)s: %(message)s', level=logging.INFO, datefmt='%I:%M:%S')
sys.path.append('../')  # Add the parent directory to the system path

import utils.haystack_utils as haystack_utils
from sparse_coding.train_autoencoder import AutoEncoder
from utils.autoencoder_utils import custom_forward, AutoEncoderConfig, evaluate_autoencoder_reconstruction, get_encoder_feature_frequencies, load_encoder, generate_with_encoder, get_direction_ablation_hook, get_top_activating_examples_for_direction, evaluate_direction_ablation_single_prompt
import utils.haystack_utils as haystack_utils
from utils.plotting_utils import line
from utils.circuit_discovery_utils import *

%reload_ext autoreload
%autoreload 2

In [2]:
model_name = "tiny-stories-2L-33M"
model = HookedTransformer.from_pretrained(
    model_name,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    device=device,
)
model.set_use_attn_result(True)

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/323M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

Loaded pretrained model tiny-stories-2L-33M into HookedTransformer


In [3]:
# L0 '2_silvery_smoke',
# l1 '2_soft_monkey',  
# L3 '2_driven_planet'
run_names = ["18_morning_sun", "8_deep_brook"]
#run_names = ['1_skilled_universe', '2_driven_planet']
encoders = []
for run_name in run_names:
    encoder, cfg = load_encoder(run_name, model_name, model)
    cfg.run_name = run_name
    print(cfg.run_name, cfg.layer, cfg.l1_coeff)
    encoders.append((encoder, cfg))

18_morning_sun 0 0.0001
8_deep_brook 1 0.0003


In [4]:
prompts = load_tinystories_validation_prompts()

(INFO) 07:53:33: Loaded 21990 TinyStories validation prompts


In [5]:
def get_activations(encoder, cfg, encoder_name, save_path="/workspace"):
    path = f"{save_path}/data/{encoder_name}_activations.pkl"
    if os.path.exists(path):
        with open(path, "rb") as f:
            data = pickle.load(f)
            max_activations = data["max_activations"]
            max_activation_token_indices = data["max_activation_token_indices"]
    else:
        max_activations, max_activation_token_indices = get_max_activations(prompts, model, encoder, cfg)
        with open(path, "wb") as f:
            pickle.dump({"max_activations": max_activations, "max_activation_token_indices": max_activation_token_indices}, f)
    return max_activations, max_activation_token_indices

In [6]:
max_activation_data = {}
for encoder, cfg in encoders:
    run_name = cfg.run_name
    max_activations, max_activation_token_indices = get_activations(encoder, cfg, run_name)
    max_activation_data[run_name] = {
        "max_activations": max_activations.cpu(),
        "max_activation_token_indices": max_activation_token_indices.cpu()
    }

In [7]:
def print_top_examples(prompts: list[str], activations: Float[Tensor, "n_prompts d_enc"], direction: int, encoder: AutoEncoder, cfg: AutoEncoderConfig, n=5):
    top_idxs = activations[:, direction].argsort(descending=True)[:n].cpu().tolist()
    for prompt_index in top_idxs:
        prompt = prompts[prompt_index]
        prompt_tokens = model.to_str_tokens(model.to_tokens(prompt))
        acts = get_acts(prompt, model, encoder, cfg)
        direction_act = acts[:, direction].cpu().tolist()
        max_direction_act = max(direction_act)
        if max_direction_act > 0:
            haystack_utils.clean_print_strings_as_html(prompt_tokens, direction_act, max_value=max_direction_act)

In [8]:
first_encoder, first_encoder_cfg = encoders[0]
second_encoder, second_encoder_cfg = encoders[1]

first_encoder_max_activations = max_activation_data[first_encoder_cfg.run_name]["max_activations"]
first_encoder_max_activation_token_indices = max_activation_data[first_encoder_cfg.run_name]["max_activation_token_indices"]
second_encoder_max_activations = max_activation_data[second_encoder_cfg.run_name]["max_activations"]
second_encoder_max_activation_token_indices = max_activation_data[second_encoder_cfg.run_name]["max_activation_token_indices"]

## Pairwise cosine circuit discovery

In [ ]:
W_out = model.W_out[first_encoder_cfg.layer]
W_in = model.W_in[second_encoder_cfg.layer]

cosine_sims = torch.nn.functional.normalize(first_encoder.W_dec @ W_out, dim=-1) @ torch.nn.functional.normalize(W_in @ second_encoder.W_enc, dim=0)
cosine_sims = torch.tril(cosine_sims)

def i_to_row_col(i: int, n_cols: int = first_encoder.d_hidden):
    row = i // n_cols
    col = i % n_cols
    return row, col

all_sims = cosine_sims.flatten().cpu()
top_cosine_similarities, top_cosine_sim_indices = torch.topk(all_sims, 10)

In [ ]:
data = []
for top_cosine_index in tqdm(top_cosine_sim_indices):
    first_encoder_dir, second_encoder_dir = i_to_row_col(top_cosine_index)
    top_prompts, top_prompt_token_indices = get_top_activating_examples_for_direction(prompts, first_encoder_dir, first_encoder_max_activations, first_encoder_max_activation_token_indices, k=100)
    
    original_losses = []
    first_encoder_losses = []
    second_encoder_losses = []
    acts = []
    ablated_acts = []
    for prompt, pos in zip(top_prompts, top_prompt_token_indices.tolist()):
        # Direction losses
        original_loss, first_encoder_ablated_loss = evaluate_direction_ablation_single_prompt(prompt, first_encoder, model, first_encoder_dir, first_encoder_cfg, pos=pos)
        _, second_encoder_ablated_loss = evaluate_direction_ablation_single_prompt(prompt, second_encoder, model, second_encoder_dir, second_encoder_cfg, pos=pos)
        original_losses.append(original_loss)
        first_encoder_losses.append(first_encoder_ablated_loss)
        second_encoder_losses.append(second_encoder_ablated_loss)

        # Second encoder direction activation with and without ablation
        act = get_acts(prompt, model, second_encoder, second_encoder_cfg)[pos, second_encoder_dir].item()
        encoder_hook_point = f"blocks.{first_encoder_cfg.layer}.{first_encoder_cfg.act_name}"
        with model.hooks(fwd_hooks=[(encoder_hook_point, get_direction_ablation_hook(first_encoder, first_encoder_dir, pos))]):
            ablated_act = get_acts(prompt, model, second_encoder, second_encoder_cfg)[pos, second_encoder_dir].item()
        acts.append(act)
        ablated_acts.append(ablated_act)

    data.append([first_encoder_dir.item(), second_encoder_dir.item(), np.mean(original_losses), np.mean(first_encoder_losses), np.mean(second_encoder_losses), np.mean(acts), np.mean(ablated_acts)])
df = pd.DataFrame(data, columns=["Encoder 1 direction", "Encoder 2 direction", "Original loss", "Encoder 1 direction ablation loss", "Encoder 2 direction ablation loss", "Second encoder activation", "Second encoder activation after ablation"])
df["Cosine similarity"] = top_cosine_similarities.tolist()

## Prompt co occurrence analysis

In [ ]:
# Pick interesting looking prompt
# Save activations of all directions for that prompt
# Save last layer active directions for each earlier direction ablated individually
# Compute AND measure for all active directions in last layer based on previous layers

In [ ]:
prompt = "This moral story teaches children that"
second_encoder_acts = get_acts(prompt, model, second_encoder, second_encoder_cfg)[-1]
second_encoder_top_acts, second_encoder_top_dirs = torch.topk(second_encoder_acts, 10)
second_encoder_direction = second_encoder_top_dirs[0].item()
first_encoder_acts = get_acts(prompt, model, first_encoder, first_encoder_cfg)[-1]
active_first_encoder_directions = torch.argwhere(first_encoder_acts > 1).flatten().tolist()
original_second_encoder_act = second_encoder_top_acts[0].item()
#px.histogram(acts.cpu().numpy(), width=700)

In [ ]:
data = []
hook_point = first_encoder_cfg.encoder_hook_point
for first_encoder_direction in active_first_encoder_directions:
    ablation_hook = get_direction_ablation_hook(first_encoder, first_encoder_direction, -1)
    with model.hooks([(hook_point, ablation_hook)]):
        ablated_acts = get_acts(prompt, model, second_encoder, second_encoder_cfg)[-1, second_encoder_direction].item()
    data.append([first_encoder_direction, ablated_acts])
and_df = pd.DataFrame(data, columns=["First encoder direction", "Second encoder activation after ablation"])
and_df["Activation difference"] = and_df["Second encoder activation after ablation"] - original_second_encoder_act
and_df

In [ ]:
data = []
hook_point = first_encoder_cfg.encoder_hook_point
for prompt_index, prompt in enumerate(prompts[:2]):
    second_encoder_acts_all_pos = get_acts(prompt, model, second_encoder, second_encoder_cfg)
    first_encoder_acts_all_pos = get_acts(prompt, model, first_encoder, first_encoder_cfg)
    num_tokens = second_encoder_acts_all_pos.shape[0]
    for position in range(10, num_tokens):
        first_encoder_acts = first_encoder_acts_all_pos[position]
        second_encoder_acts = second_encoder_acts_all_pos[position]

        second_encoder_top_acts, second_encoder_top_dirs = torch.topk(second_encoder_acts, 10)
        second_encoder_direction = second_encoder_top_dirs[0].item()
        active_first_encoder_directions = torch.argwhere(first_encoder_acts > 1).flatten().tolist()
        original_second_encoder_act = second_encoder_top_acts[0].item()

        for first_encoder_direction in active_first_encoder_directions:
            ablation_hook = get_direction_ablation_hook(first_encoder, first_encoder_direction, position)
            with model.hooks([(hook_point, ablation_hook)]):
                ablated_acts = get_acts(prompt, model, second_encoder, second_encoder_cfg)[position, second_encoder_direction].item()
            data.append([prompt_index, position, first_encoder_direction, second_encoder_direction, ablated_acts, original_second_encoder_act])
and_df = pd.DataFrame(data, columns=["Prompt", "Position", "First encoder direction", "Second encoder direction", "Second encoder activation after ablation", "Second encoder activation"])
and_df["Activation difference"] = and_df["Second encoder activation after ablation"] - and_df["Second encoder activation"]
and_df = and_df.sort_values("Activation difference", ascending=False)

In [ ]:
and_df.head(10)

In [ ]:
print_top_examples(prompts, second_encoder_max_activations, second_encoder_direction, second_encoder, second_encoder_cfg, n=1)

## Get top token occurrences per direction

In [ ]:
def get_direction_token_df(max_activations, prompts, model, encoder, encoder_cfg, percentage_threshold=0.5, save_path="/workspace/data/top_token_occurrences"):
    os.makedirs(save_path, exist_ok=True)
    file_name = f"{save_path}/{encoder_cfg.run_name}_direction_token_occurrences.csv"
    if os.path.exists(file_name):
        direction_df = pd.read_csv(file_name)
    else:

        token_wise_activations = eval_direction_tokens_global(max_activations, prompts, model, encoder, encoder_cfg, percentage_threshold=0.5)
        total_occurrences = token_wise_activations.sum(1)
        max_occurrences = token_wise_activations.max(1)[0]
        max_occurring_token = token_wise_activations.argmax(1)
        str_tokens = model.to_str_tokens(torch.LongTensor(list(range(model.cfg.d_vocab))))

        direction_data = []
        for direction in tqdm(range(encoder.d_hidden)):
            total_occurrence = total_occurrences[direction].item()
            top_occurrence = max_occurrences[direction].item()
            top_token = model.to_single_str_token(max_occurring_token[direction].item())
            direction_data.append([direction, total_occurrence, top_token, top_occurrence])

        direction_df = pd.DataFrame(direction_data, columns=["Direction", "Total occurrences", "Top token", "Top token occurrences"])
        direction_df["Top token percent"] = direction_df["Top token occurrences"] / direction_df["Total occurrences"]
        direction_df = direction_df.dropna()

    print(len(direction_df))
    return direction_df

direction_df = get_direction_token_df(first_encoder_max_activations, prompts, model, first_encoder, first_encoder_cfg, percentage_threshold=0.5)

In [ ]:
fig = px.histogram(direction_df, x="Top token percent", width=700, title="Per direction percentage of activations on top token")
fig.update_layout({
    "xaxis_title": "Top token activation percentage",
})
fig.show()

In [ ]:
good_directions = direction_df[(direction_df["Top token percent"] > 0.2) & (direction_df["Top token percent"] < 0.7)]["Direction"].tolist()
#good_directions = direction_df[(direction_df["Top token percent"] < 0.1)]["Direction"].tolist()

print(len(good_directions))

In [ ]:
def print_top_examples(prompts: list[str], activations: Float[Tensor, "n_prompts d_enc"], direction: int, encoder: AutoEncoder, cfg: AutoEncoderConfig, n=5):
    top_idxs = activations[:, direction].argsort(descending=True)[:n].cpu().tolist()
    max_direction_act = activations[:, direction].max().item()
    for prompt_index in top_idxs:
        prompt = prompts[prompt_index]
        prompt_tokens = model.to_str_tokens(model.to_tokens(prompt))
        acts = get_acts(prompt, model, encoder, cfg)
        direction_act = acts[:, direction].cpu().tolist()
        if max(direction_act) > 0:
            haystack_utils.clean_print_strings_as_html(prompt_tokens, direction_act, max_value=max_direction_act)

def print_direction_example(direction, n=10):
    print_top_examples(prompts, first_encoder_max_activations, direction, first_encoder, first_encoder_cfg, n)

# Max activations
_ = interact(print_direction_example, 
         direction=SelectionSlider(options=good_directions, value=good_directions[0], description='Direction'),
         #direction=IntSlider(min=0, max=l0_encoder.d_hidden-1, step=1, value=0),
         n=IntSlider(min=1, max=20, step=1, value=5))

## Quotation mark

In [9]:
# Capabilities
# Start quotation after "said" or ":" or other obvious tokens (check tokenization)
# End quotation after "." if started



In [10]:
# Model can definitely predict 'said, "' trigram
#test_prompt(test_prompts[1][:-2], " \"", model)

In [71]:
# '."' '?"' and '!"' are single tokens
# Save prompts where next token is '."'
test_prompts = []
activating_test_prompts = []
for prompt in tqdm(prompts):
    if "said, \"" in prompt:
        start_index = prompt.index("said, \"") + 7
        end_index = prompt.find(".\"", start_index)
        if end_index != -1:
            subprompt = prompt[:end_index+2]
            tokens = model.to_tokens(subprompt)
            last_token = model.to_single_str_token(tokens[0, -1].item())
            if (subprompt[-2:] == ".\"") and (last_token == ".\""):
                test_prompts.append(subprompt)
                act_token_index = tokens.shape[1] - 1
                act = get_acts(subprompt, model, second_encoder, second_encoder_cfg)[act_token_index, direction].item()
                if act > 0.1:
                    activating_test_prompts.append(subprompt)
print(len(test_prompts), len(activating_test_prompts))
print(test_prompts[0])

  0%|          | 0/21990 [00:00<?, ?it/s]

4554 12
Spot. Spot saw the shiny car and said, "Wow, Kitty, your car is so bright and clean!" Kitty smiled and replied, "Thank you, Spot. I polish it every day."


In [12]:
test_prompt(test_prompts[0][:-2], '."', model, prepend_space_to_answer=False)

Tokenized prompt: ['<|endoftext|>', 'Spot', '.', ' Spot', ' saw', ' the', ' shiny', ' car', ' and', ' said', ',', ' "', 'Wow', ',', ' Kitty', ',', ' your', ' car', ' is', ' so', ' bright', ' and', ' clean', '!"', ' Kitty', ' smiled', ' and', ' replied', ',', ' "', 'Thank', ' you', ',', ' Spot', '.', ' I', ' polish', ' it', ' every', ' day']
Tokenized answer: ['."']


Performance on answer token:
Rank: 1        Logit: 24.84 Prob: 30.33% Token: |."|

Top 0th token. Logit: 25.26 Prob: 46.09% Token: | to|
Top 1th token. Logit: 24.84 Prob: 30.33% Token: |."|
Top 2th token. Logit: 22.82 Prob:  4.03% Token: |,|
Top 3th token. Logit: 22.62 Prob:  3.30% Token: | so|
Top 4th token. Logit: 22.54 Prob:  3.05% Token: |.|
Top 5th token. Logit: 22.49 Prob:  2.88% Token: | and|
Top 6th token. Logit: 22.45 Prob:  2.77% Token: | with|
Top 7th token. Logit: 22.26 Prob:  2.31% Token: |!"|
Top 8th token. Logit: 21.80 Prob:  1.45% Token: | because|
Top 9th token. Logit: 21.30 Prob:  0.88% Token: | for|


Ranks of the answer tokens: [('."', 1)]

In [13]:
def DLA(prompts: list[str], model: HookedTransformer, pos=-1) -> tuple[Float[Tensor, "component"], list[str]]:
    logit_attributions = []
    for prompt in tqdm(prompts):
        tokens = model.to_tokens(prompt)
        answers = tokens[:, 1:]
        tokens = tokens[:, :-1]
        answer_residual_directions = model.tokens_to_residual_directions(answers)[:, pos]  # [batch pos d_model]
        _, cache = model.run_with_cache(tokens)
        accumulated_residual, labels = cache.get_full_resid_decomposition(layer=-1, pos_slice=pos, return_labels=True, expand_neurons=False)
        scaled_residual_stack = cache.apply_ln_to_stack(accumulated_residual, layer = -1, pos_slice=pos)
        logit_attribution = einops.einsum(scaled_residual_stack, answer_residual_directions, "component batch d_model, batch d_model -> component") / answers.shape[1]
        logit_attributions.append(logit_attribution)
    
    logit_attributions = torch.stack(logit_attributions).mean(0)
    return logit_attributions, labels

In [14]:
# DLA of '."', find relevant MLPs if they exist, check encoders of those MLPs for relevant directions
# DLA looks pretty different for different prompts
# Relevant boosting heads:
# Relevant boosting MLP: 
# Relevant deboosting heads: 
# Relevant deboosting MLP:
dlas, labels = DLA(test_prompts, model)
print(dlas.shape)

  0%|          | 0/132 [00:00<?, ?it/s]

torch.Size([37])


In [57]:
line(dlas.cpu().numpy(), xticks=labels, width=1200, title="Closing quotation DLA", show_legend=False)

In [22]:
second_encoder_cfg

AutoEncoderConfig(layer=1, act_name='mlp.hook_post', expansion_factor=4, l1_coeff=0.0003, d_in=4096, run_name='8_deep_brook')

In [23]:
all_acts = []
for prompt in test_prompts:
    acts = get_acts(prompt, model, second_encoder, second_encoder_cfg)[-2]
    all_acts.append(acts)

all_acts = torch.stack(all_acts).mean(0)
top_acts, top_dirs = torch.topk(all_acts, 10)
print(top_acts, top_dirs)

tensor([1.0361, 0.3616, 0.3409, 0.2595, 0.2429, 0.2298, 0.2251, 0.1967, 0.1930,
        0.1760], device='cuda:0') tensor([15330,  7758,  3373,  7447,  2594,  8842, 10040, 14277,  1790, 11856],
       device='cuda:0')


In [24]:
fig = px.histogram(all_acts.cpu().numpy(), width=700)


## Direction 2236

In [25]:
direction = 15330#2236
# Max examples
# Token activations
# Token boosts
# Feature DDA (direct direction attribution)

In [26]:
# Activates on many different tokens within quotation marks    
# Higher activations seem to correspond to possible endings of quotation marks
print_top_examples(prompts, second_encoder_max_activations, direction, second_encoder, second_encoder_cfg, n=5)

In [27]:
def get_common_tinystories_tokens(prompts, model: HookedTransformer, min_occurrences=100):
    occurrences = torch.zeros(model.cfg.d_vocab, dtype=torch.int32).cuda()
    for prompt in prompts: 
        tokens = model.to_tokens(prompt).flatten()
        occurrences = occurrences.index_add(0, tokens, torch.ones_like(tokens, dtype=torch.int32))
    common_tokens = torch.argwhere(occurrences > min_occurrences).flatten()
    rare_tokens = torch.argwhere(occurrences <= min_occurrences).flatten()
    return occurrences, common_tokens, rare_tokens

occurrences, common_tokens, rare_tokens = get_common_tinystories_tokens(prompts, model)
print(len(common_tokens), len(rare_tokens))

2552 47705


In [28]:
quotation_tokens = ["\"", ".", "!", "?", ".\"", "!\"", "?\""]
for quotation_token in quotation_tokens:
    assert model.to_single_token(quotation_token) in common_tokens

answer_token = model.to_single_token(".\"")
print(answer_token, model.to_single_str_token(answer_token))

526 ."


In [29]:
# Prompt based token boosts

def get_direction_logit_and_logprob_boost(
    prompt: str,
    encoder: AutoEncoder,
    encoder_neuron,
    model: HookedTransformer,
    all_ignore: Int[Tensor, "tokens"],
    cfg: AutoEncoderConfig,
    pos: -2
):
    zero_direction_hook = [(f"blocks.{cfg.layer}.{cfg.act_name}", get_direction_ablation_hook(
        encoder, encoder_neuron, pos
    ))]

    logits_active = model(prompt, return_type="logits")[0, pos]
    with model.hooks(zero_direction_hook):
        logits_inactive = model(prompt, return_type="logits")[0, pos]

    logprobs_active = logits_active.log_softmax(dim=-1)
    logprobs_inactive = logits_inactive.log_softmax(dim=-1)
    print(logprobs_active[answer_token], logprobs_inactive[answer_token])

    boosts = (logprobs_active - logprobs_inactive)
    boosts[logprobs_active < -7] = 0
    boosts[all_ignore] = 0
    top_boosts, top_tokens = torch.topk(boosts, 15)
    non_zero_boosts = top_boosts != 0
    top_deboosts, top_deboosted_tokens = torch.topk(boosts, 15, largest=False)
    non_zero_deboosts = top_deboosts != 0
    boosted_tokens = (
        model.to_str_tokens(top_tokens[non_zero_boosts]),
        top_boosts[non_zero_boosts].tolist(),
    )
    deboosted_tokens = (
        model.to_str_tokens(top_deboosted_tokens[non_zero_deboosts]),
        top_deboosts[non_zero_deboosts].tolist(),
    )
    logging.info(f"Top boosted: {boosted_tokens}")
    logging.info(f"Top deboosted: {deboosted_tokens}")

get_direction_logit_and_logprob_boost(test_prompts[3], second_encoder, direction, model, rare_tokens, second_encoder_cfg, pos=-2)

(INFO) 08:03:40: Top boosted: (['!"', '."'], [0.2043628692626953, 0.08043524622917175])
(INFO) 08:03:40: Top deboosted: (['.', ' but', ',', ' and', ' like', ' to'], [-0.46565985679626465, -0.3169403076171875, -0.29815053939819336, -0.2541923522949219, -0.22343969345092773, -0.13779401779174805])


tensor(-0.2608, device='cuda:0') tensor(-0.3413, device='cuda:0')


In [30]:
# Weight based token boosts

def get_direction_boosted_tokens(direction, encoder: AutoEncoder, model: HookedTransformer, cfg: AutoEncoderConfig, rare_tokens: Tensor):
    token_boosts = encoder.W_dec[direction] @ model.W_out[cfg.layer] @ model.unembed.W_U
    token_boosts[rare_tokens] = 0
    return token_boosts

def print_token_boosts(boosts, tokens):
    str_tokens = model.to_str_tokens(tokens)
    boost_str = ""
    for token, boost in zip(str_tokens, boosts.tolist()):
        boost_str += f"('{token}': {boost:.2f}), "
    print(boost_str[:-2])

boosts = get_direction_boosted_tokens(direction, second_encoder, model, second_encoder_cfg, rare_tokens)
top_boosts, top_tokens = torch.topk(boosts, 25)
print_token_boosts(top_boosts, top_tokens)
top_boosts, top_tokens = torch.topk(boosts, 25, largest=False)
print_token_boosts(top_boosts, top_tokens)

('!"': 0.57), ('."': 0.41), ('!".': 0.37), (' next': 0.34), (' today': 0.32), (' now': 0.28), (' here': 0.28), (' because': 0.26), (' last': 0.24), (' too': 0.24), (' I': 0.24), ('".': 0.23), (' right': 0.23), (' soon': 0.23), (' down': 0.22), (' we': 0.22), (' my': 0.22), (' on': 0.21), (' behind': 0.20), (' sitting': 0.20), (' outside': 0.20), (' ahead': 0.20), (' holding': 0.20), (' anyway': 0.20), (' to': 0.19)
('?': -0.39), ('
': -0.39), ('!': -0.30), (' ': -0.26), (' stove': -0.22), ('uck': -0.22), (' She': -0.22), (' repair': -0.21), (' she': -0.20), (' herself': -0.20), (' dream': -0.20), (' couldn': -0.20), (' comet': -0.20), (' her': -0.19), (' drank': -0.19), (' dive': -0.19), ('P': -0.19), (' colourful': -0.19), ('D': -0.19), (' imagined': -0.19), (' bath': -0.19), ('ag': -0.19), (' fire': -0.19), ('ind': -0.19), (' refused': -0.18)


In [62]:
prompt = test_prompts[0]
print(prompt)
test_prompt('Spot. Spot saw the shiny car and said, "Wow, Kitty, your car is so bright and clean!" Kitty smiled and replied, "Thank you, Spot. I polish it every day', '.\"', model, prepend_space_to_answer=False)

Spot. Spot saw the shiny car and said, "Wow, Kitty, your car is so bright and clean!" Kitty smiled and replied, "Thank you, Spot. I polish it every day."
Tokenized prompt: ['<|endoftext|>', 'Spot', '.', ' Spot', ' saw', ' the', ' shiny', ' car', ' and', ' said', ',', ' "', 'Wow', ',', ' Kitty', ',', ' your', ' car', ' is', ' so', ' bright', ' and', ' clean', '!"', ' Kitty', ' smiled', ' and', ' replied', ',', ' "', 'Thank', ' you', ',', ' Spot', '.', ' I', ' polish', ' it', ' every', ' day']
Tokenized answer: ['."']


Performance on answer token:
Rank: 1        Logit: 24.84 Prob: 30.33% Token: |."|

Top 0th token. Logit: 25.26 Prob: 46.09% Token: | to|
Top 1th token. Logit: 24.84 Prob: 30.33% Token: |."|
Top 2th token. Logit: 22.82 Prob:  4.03% Token: |,|
Top 3th token. Logit: 22.62 Prob:  3.30% Token: | so|
Top 4th token. Logit: 22.54 Prob:  3.05% Token: |.|
Top 5th token. Logit: 22.49 Prob:  2.88% Token: | and|
Top 6th token. Logit: 22.45 Prob:  2.77% Token: | with|
Top 7th token. Logit: 22.26 Prob:  2.31% Token: |!"|
Top 8th token. Logit: 21.80 Prob:  1.45% Token: | because|
Top 9th token. Logit: 21.30 Prob:  0.88% Token: | for|


Ranks of the answer tokens: [('."', 1)]

In [79]:
# Does ablating the feature increase loss when the feature is active?

# On positions which close quotation
loss_increases = []
for prompt in activating_test_prompts:
    tokens = model.to_tokens(prompt)
    pos = tokens.shape[1]-2
    original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(tokens, second_encoder, model, direction, second_encoder_cfg, pos=pos)
    loss_increase = ablated_loss - original_loss
    loss_increases.append(loss_increase)
print(np.mean(loss_increases), np.std(loss_increases))

1.954936146736145 1.9467954635620117
0.17658671736717224 0.1686093509197235
0.21852493286132812 0.21843621134757996
0.029260706156492233 0.029260706156492233
0.7188500165939331 0.739274799823761
1.1689330339431763 1.1427963972091675
1.175621747970581 1.1732035875320435
1.402824878692627 1.402824878692627
0.14515602588653564 0.14515602588653564
0.1105048730969429 0.10319109261035919
2.7097363471984863 2.7971956729888916
0.6739743947982788 0.6739743947982788
0.0046507300188144045 0.026893514700522085


In [78]:
# In general on max activating prompts, ablating globally
loss_increases = []
max_activating_prompts, max_activating_token_indices = get_top_activating_examples_for_direction(prompts, direction, second_encoder_max_activations, second_encoder_max_activation_token_indices, k=100)
for prompt, index in zip(max_activating_prompts, max_activating_token_indices.tolist()):
    original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(prompt, second_encoder, model, direction, second_encoder_cfg)
    loss_increase = ablated_loss - original_loss
    loss_increases.append(loss_increase)
print(np.mean(loss_increases), np.std(loss_increases))

0.001261785626411438 0.0029002849547573144


In [33]:
# In general on max activating prompts, ablating one activating position at a time
loss_increases = []
max_activating_prompts, max_activating_token_indices = get_top_activating_examples_for_direction(prompts, direction, second_encoder_max_activations, second_encoder_max_activation_token_indices, k=100)
threshold = second_encoder_max_activations[:, direction].max() * 0.1

for prompt, index in tqdm(zip(max_activating_prompts, max_activating_token_indices.tolist()), total=len(max_activating_prompts)):
    acts = get_acts(prompt, model, second_encoder, second_encoder_cfg)[:, direction]
    # No loss for last position, exclude
    active_positions = torch.argwhere(acts[:-1] > threshold).flatten().tolist()
    for position in active_positions:
        original_loss, ablated_loss = evaluate_direction_ablation_single_prompt(prompt, second_encoder, model, direction, second_encoder_cfg, pos=position)
        loss_increase = ablated_loss - original_loss
        loss_increases.append(loss_increase)
print(np.mean(loss_increases), np.std(loss_increases))

  0%|          | 0/100 [00:00<?, ?it/s]

0.005296133676435087 0.22038943811179693


## Direction 15777 (second highest quotation DLA)

In [34]:
direction = 15777

In [35]:
print_top_examples(prompts, second_encoder_max_activations, direction, second_encoder, second_encoder_cfg, n=5)

In [36]:
get_direction_logit_and_logprob_boost(test_prompts[3], second_encoder, direction, model, rare_tokens, second_encoder_cfg, pos=-2)

(INFO) 08:04:06: Top boosted: ([], [])
(INFO) 08:04:06: Top deboosted: ([], [])


tensor(-0.2608, device='cuda:0') tensor(-0.2608, device='cuda:0')


## Check for directions with positive DLA


In [45]:
(all_acts.unsqueeze(1) * second_encoder.W_dec)

tensor(-0.0070, device='cuda:0')

In [58]:
activation_scaled_direction_dla = (all_acts.unsqueeze(1) * second_encoder.W_dec) @ model.W_out[second_encoder_cfg.layer] @ model.unembed.W_U[:, answer_token]
print(activation_scaled_direction_dla.shape)
top_acts, top_dirs = torch.topk(activation_scaled_direction_dla, 10)
print(top_acts, top_dirs)

torch.Size([16384])
tensor([0.4200, 0.1860, 0.1287, 0.1172, 0.1156, 0.1002, 0.0894, 0.0833, 0.0822,
        0.0683], device='cuda:0') tensor([15330,  3373,  7447,  2594,  8842, 11856,  6829, 14277, 14727,  8915],
       device='cuda:0')


In [48]:
direction_dla = second_encoder.W_dec @ model.W_out[second_encoder_cfg.layer] @ model.unembed.W_U[:, answer_token]
print(direction_dla.shape)

torch.Size([16384])


In [50]:
active_directions = (all_acts > 0.05).cpu()
print(active_directions.sum())

fig = go.Figure(data=go.Scatter(x=direction_dla.cpu()[active_directions], y=all_acts.cpu()[active_directions], mode='markers'))
fig.update_layout(
    title="Gradients of decoder features in Layer 0 wrt a Layer 1 feature against cosine similarities of same",
    xaxis_title='Direction \'.\"\' DLA',
    yaxis_title='Direction activation',
    width=900
)
fig.show()

tensor(87)


In [52]:
active_directions = (all_acts > 0.05).cpu()
print(active_directions.sum())

fig = go.Figure(data=go.Scatter(x=activation_scaled_direction_dla.cpu()[active_directions], y=all_acts.cpu()[active_directions], mode='markers'))
fig.update_layout(
    title="DLA and activation of active Layer 1 encoder directions",
    xaxis_title='Direction \'.\"\' DLA',
    yaxis_title='Direction activation',
    width=900
)
fig.show()

tensor(87)
